<a href="https://colab.research.google.com/github/UniVR-DH/DBMS-course/blob/main/notebooks/lab03-sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL exercise with SQLite in Jupyter Notebooks
In this notebook we use SQLite as a DBMS, plus we use some plugins to simplify your way to run SQL queries.

## Library Import and Configuration

In [1]:
!pip install --quiet jupysql
!pip install --quiet pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00


In [2]:
import sqlite3
import pandas as pd
# Import jupysql Jupyter extension to create SQL cells
# this avoids the need to run SQL in python
%load_ext sql

**We configure jupysql to return data as a Pandas dataframe and have less verbose output**

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Initialize the Database

In [4]:
# Run SQLite saving data to a file called `mydb.db` -- you can change the name
%sql sqlite:///mydb.db

An entire Jupyter cell can be used as a SQL cell by placing `%%sql` at the start of the cell. Query results will be displayed as a Pandas DF.

In [5]:
%%sql
SELECT 1=2 as test, 'Hello people' as message, 3*12345 as math  ;

,test,message,math
0,0,Hello people,37035


## Create tables with some data

In [6]:
%%sql
CREATE TABLE customer (
    id INTEGER PRIMARY KEY,
    name varchar(25)
);


CREATE TABLE c_order (
    ordid INTEGER PRIMARY KEY,
    date DATE,
    cid INTEGER DEFAULT 0,
    FOREIGN KEY (cid) REFERENCES customer(id)
);


CREATE TABLE invoice (
    ordid INTEGER,
    price INTEGER,
    cid INTEGER DEFAULT 0,
    part INTEGER DEFAULT 0,
    FOREIGN KEY (ordid) REFERENCES customer_order(ordid),
    FOREIGN KEY (cid) REFERENCES customer(id)
    PRIMARY KEY (part, ordid)
);

""


In [7]:
%%sql
INSERT INTO customer (id, name)
VALUES
(2101, "Christian"),
(2102, "Jens"),
(2103, "Kristian"),
(2104, "Christian");


INSERT INTO c_order (ordid, date, cid)
VALUES
(97001, '2024-10-1', 2101),
(97002, '2024-06-1', 2101),
(97003, '2023-11-1', 2103),
(97004, '2023-11-1', 0)
;


INSERT INTO invoice (ordid, price, cid, part)
VALUES
(97001, 100, 2101, 1),
(97001, 100, 2101, 2),
(97003, 90, 2103, 1),
(10004, 10, 0, 1),
(10005, 10, 2103, 1)
;

""


## Ex1: For all invoices if they have an order print the date, if they have a customer print the name


In [8]:
%%sql
SELECT i.ordid, o.date, c.name
FROM invoice i
LEFT JOIN c_order o ON i.ordid = o.ordid
LEFT JOIN customer c ON i.cid = c.id

,ordid,date,name
0,97001,2024-10-1,Christian
1,97001,2024-10-1,Christian
2,97003,2023-11-1,Kristian
3,10004,None,None
4,10005,None,Kristian


## Ex2: Find invoices without orders and customers

In [ ]:
%%sql
SELECT *
FROM invoice i
LEFT JOIN c_order o ON i.ordid = o.ordid
LEFT JOIN customer c ON i.cid = c.id
WHERE o.ordid IS NULL AND c.id IS NULL

,ordid,price,cid,part,ordid,date,cid,id,name
0,10004,10,0,1,None,None,None,None,None


In [ ]:
%%sql
SELECT *
FROM invoice i
WHERE i.ordid NOT IN  (SELECT ordid FROM c_order)
     AND i.cid NOT IN (SELECT id FROM customer)

,ordid,price,cid,part
0,10004,10,0,1


## Ex3: Find the total amount owed by each customer


In [16]:
%%sql
INSERT INTO invoice (ordid, price, cid, part)
VALUES
(7701, 0, 2104, 1)
;

""


In [17]:
%%sql
SELECT c.id, c.name, i.price A
FROM customer c
LEFT JOIN invoice i ON c.id = i.cid


,id,name,A
0,2101,Christian,100
1,2101,Christian,100
2,2102,Jens,-100
3,2102,Jens,100
4,2103,Kristian,10
5,2103,Kristian,90
6,2104,Christian,0


In [18]:
%%sql
SELECT c.id, c.name, SUM(i.price) AS total_owed
FROM customer c
LEFT JOIN invoice i ON c.id = i.cid
GROUP BY c.id, c.name;

,id,name,total_owed
0,2101,Christian,200
1,2102,Jens,0
2,2103,Kristian,100
3,2104,Christian,0


## Ex4: Find the total amount owed by each customer inlcuding unknown




In [19]:
%%sql
SELECT
  c.name,
  SUM(i.price) AS total_owed
FROM invoice AS i
LEFT JOIN customer AS c ON i.cid = c.id
GROUP BY c.name;

,name,total_owed
0,None,10
1,Christian,200
2,Jens,0
3,Kristian,100


## Ex5: Find the total amount owed by each customer inlcuding unknown, grouped by order id

In [22]:
%%sql
SELECT
c.name,
  ordid,
  SUM(i.price) AS total_owed
FROM invoice AS i
LEFT JOIN customer AS c ON i.cid = c.id
GROUP BY  c.name, ordid;


,name,ordid,total_owed
0,None,10004,10
1,Christian,7701,0
2,Christian,97001,200
3,Jens,77001,0
4,Kristian,10005,10
5,Kristian,97003,90


# Exercises Lecture 12

In [23]:
%%sql
CREATE TABLE professor (
    id INTEGER PRIMARY KEY,
    name TEXT,
    rank TEXT,
    salary INTEGER
);

INSERT INTO professor (id, name, rank, salary) VALUES
(2125, 'Socrates', 'C4', 100),
(2126, 'Russel', 'C4', 100),
(2134, 'Augustinus', 'C3', 200),
(2137, 'Kant', 'C4', 200),
(2140, 'Magnus', 'C2', 200),
(2141, 'Eco', 'C9', 1000);


CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    title TEXT,
    ects INTEGER,
    students INTEGER,
    by INTEGER
);

INSERT INTO course (id, title, ects, students, by) VALUES
(1, 'Basic', 4, 20, 2137),
(2, 'Ethics', 4, 10, 2125),
(3, 'Theory Cogn', 3, 50, 2126),
(4, 'CS 101', 2, 40, 2134),
(5, 'Math', 8, 20, 2137),
(6, 'Morals', 3, 20, 2140),
(7, 'Algebra', 3, 20, 2126),
(8, 'Algos', 6, 20, 2125),
(9, 'OS', 2, 100, 2125);



""


In [25]:
%%sql
-- GQ1
SELECT p.id, p.name, SUM(c.students), AVG(ects), COUNT(*)
FROM professor p
     JOIN course c ON p.id = c.by
GROUP BY p.id, p.name
HAVING AVG(ects) > 2;

,id,name,SUM(c.students),AVG(ects),COUNT(*)
0,2125,Socrates,130,4.0,3
1,2126,Russel,70,3.0,2
2,2137,Kant,40,6.0,2
3,2140,Magnus,20,3.0,1


In [29]:
%%sql
-- GQ+
SELECT p.id, p.name,(c.students), (ects)
FROM professor p
     JOIN course c ON p.id = c.by



,id,name,students,ects
0,2137,Kant,20,4
1,2125,Socrates,10,4
2,2126,Russel,50,3
3,2134,Augustinus,40,2
4,2137,Kant,20,8
5,2140,Magnus,20,3
6,2126,Russel,20,3
7,2125,Socrates,20,6
8,2125,Socrates,100,2


## Compare GQ2  with GQ3 what is going on?

In [26]:
%%sql
-- GQ2
SELECT p.id, p.name, SUM(c.students)
FROM professor p
     JOIN course c   ON p.id = c.by
GROUP BY p.id, p.name
HAVING c.ects > 3;


,id,name,SUM(c.students)
0,2125,Socrates,130
1,2137,Kant,40


In [ ]:
%%sql
-- GQ3
SELECT p.id, p.name, SUM(c.students)
FROM professor p
     JOIN course c   ON p.id = c.by
WHERE c.ects > 3
GROUP BY p.id, p.name;

,id,name,SUM(c.students)
0,2125,Socrates,30
1,2137,Kant,40


In [ ]:
%%sql
-- GQ4
SELECT p.id, p.name, SUM(c.students)
FROM professor p
      JOIN course c ON p.id = c.by
GROUP BY p.id, p.name
HAVING COUNT(*) < 2


,id,name,SUM(c.students)
0,2134,Augustinus,40.0
1,2140,Magnus,20.0
2,2141,Eco,NaN


In [30]:
%%sql
--  GQ5
SELECT p.id, p.name, p.salary, SUM(c.students)
FROM professor p
     LEFT JOIN course c ON p.id = c.by
               AND c.ects > 2
GROUP BY p.id, p.name, p.salary
HAVING p.salary > 100


,id,name,salary,SUM(c.students)
0,2134,Augustinus,200,NaN
1,2137,Kant,200,40.0
2,2140,Magnus,200,20.0
3,2141,Eco,1000,NaN


In [ ]:
%%sql